# Project

In [12]:
import pandas as pd

In [23]:
path = "D:/OneDrive - University of Warwick/Dissertation/review-Alaska_10.json"

data_frame = pd.read_json(path, lines=True, encoding='utf-8')

c:\Python38\lib\site-packages\pandas\core\dtypes\cast.py:995: RuntimeWarning: invalid value encountered in cast
  return arr.astype(dtype, copy=True)


This indicates the presence of NA values in the data. However, as this is one of the aspects we intend to investigate, we will selectively perform data cleaning at a later stage.

In [17]:
print(data_frame.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1608 entries, 543 to 520376
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  1608 non-null   float64
 1   name     1608 non-null   object 
 2   time     1608 non-null   int64  
 3   rating   1608 non-null   int64  
 4   text     1608 non-null   object 
 5   pics     1608 non-null   object 
 6   resp     1608 non-null   object 
 7   gmap_id  1608 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 113.1+ KB
None


In [18]:
print(data_frame.head())

          user_id                            name           time  rating  \
543  1.018101e+20                        Samantha  1621135489200       5   
544  1.168173e+20  Christian Rose Sande-Mandregan  1620128373080       5   
546  1.151922e+20                             C S  1620517202933       5   
547  1.152884e+20                       Chick Zee  1613024620459       5   
652  1.155175e+20                         Josh N.  1534545553526       5   

                                                  text  \
543  We had a bulk of laundry to wash and came acro...   
544  Excellent is the rating I give Quantum Laundry...   
546  This is the first time I have had to use a pub...   
547  Went to their soft opening today and it's alre...   
652                                        Good to go.   

                                                  pics  \
543  [{'url': ['https://lh5.googleusercontent.com/p...   
544  [{'url': ['https://lh5.googleusercontent.com/p...   
546  [{'url': ['http

## Data Dictionary

## Data Cleaning